### Week 3: Real Time Data, Understanding Orders, and Arbitrage

Todo: talk about websockets.

Todo: talk about order types (market, limit, stop loss). 

Todo: talk about quotes (bid-ask spread, order size).

In [5]:
from config import *
import websocket
import json
import numpy as np

SOCKET_ENDPOINT = "wss://data.alpaca.markets/stream"
STOCKS = ["T.AAPL"]  
# T. is trades (ie all trades executed by exchanges)  Q. is quotes (what people are willing to buy/sell for)

In [6]:
# all websockets take 3 mandatory functions: on open, on message, and on close.  
# On open gets called once when we try to connect 
# on message gets called each time we get a new piece of data sent to us
# on close gets called when we close the connection (stop listening to data)


def on_open(ws):
    # we have to send an authentication message
    auth_data = {
        "action": "authenticate",
        "data": {
            "key_id": f"{KEY}",
            "secret_key": f"{SECRET_KEY}"
        }
    }
    # authenticate our connection
    ws.send(json.dumps(auth_data))

    channel_data = {
        "action": "listen",
        "data": {"streams": STOCKS}  # streams is what channels we want to subscribe to
    }

    ws.send(json.dumps(channel_data))
    
    
def on_message(ws, message):
    # every time a message gets sent to us we have to define what to do.  In this instance we'll just print it
    # but we can get more complex by doing stuff like analyzing the high frequency data to make orders
    print("received a message")
    print(message)
    
def on_close(ws):
    print("closed connection")

In [8]:
ws = websocket.WebSocketApp(SOCKET_ENDPOINT, on_open=on_open, on_message=on_message, on_close=on_close)
# you'll want to run this in a local terminal for it to work
ws.run_forever()

received a message
{"stream":"authorization","data":{"action":"authenticate","status":"unauthorized"}}
received a message
{"stream":"authorization","data":{"action":"listen","status":"unauthorized"}}
closed connection


True

### So what do we do with this data?

Answer: By getting data almost instantly (10ish milliseconds latency - but this will differ by location) from exchanges, we're among the first people in the markets to know what people are doing.  

Here are three key strategies that "high frequency traders" use to make money.  We'll illustrate them with examples then get a bit more technical.  

### Market Making: 
Say you go to a farmer's market and want to buy some apples.  You see that farmers are selling apples for 1.00 (bid price) a piece, but suppose you heard people waiting in line to get in saying they're willing to buy apples for 1.05 (ask price) a piece.  With this information, you run ahead of everyone else and buy the apples for they're market price of 1.00 then run back to the people in line and sell your apples for 1.05 a piece.  In essense you are able to act faster than everyone else and use that to your advantage.  By doing so you're rewarded with a risk-free(ish) profit of 5 cents a peice

In the real market scenario, you have algorithms watching quotes from exchanges and if there is a sufficiently large bid ask spread, you'll purchase the shares for the bid price then immediately sell them at the ask price, being rewarded with the spread.  The issue here is that if someone sells the shares to that third person faster than you, then you've got shares you don't want to have and no one to buy them from you.

### Arbitrage: 
Say you are at the same farmers market and can purchase apples for 1.00 a piece, but you know there's another famers market across town with people buying apples at 1.20 a peice.  You purchase the apples, drive across town, then dump your apples.

In a trading scenario, you can purchase shares of Apple stock at various stock exchanges, say NYSE and Hong Kong Stock Exchange.  We would expect these prices to be very close, but occasionally the price may differ by more than a few cents.  Then we can purchase shares at the cheeper exchange then sell them at the more expensive exchange and pocket the spread. 

### Trade Following (more falls into statarb):
I can't think of a good example for this, so we'll go straight into the market example.  

Say we're watching trade streams from an exchange and notice that a huge amount of shares are trying to be sold, which might be a big bank liquidating their position.  We believe that this liquidation will cause a quick dip in share price (higher supply, same demand), so we quickly short the stock and hope the price does what we want.

### We'll code up a quick market maker algo

In [9]:
import json
from setup_requests import make_order
# the idea here is that we'll watch quote streams then try to buy at the bid price and sell at the ask price


def analyze(ws, message):
    json_dict = json.loads(message)
    quote = json_dict["data"]
    if quote["ev"] == "Q":
        asset = quote["T"]
        bid_price = quote["p"]
        bid_size = quote["s"]
        ask_price = quote["P"]
        ask_size = quote["S"]
        # naive approach: buy if bid-ask spread > $.03

        spread = quote["P"] - quote["p"]
        # we want a reasonably large spread so we make good money for the risk
        if spread >= .05 and bid_size > ask_size:
            order_size = 1
            buy_order = make_order(asset, order_size, "buy", "limit", "fok", ask_price)
            if buy_order["status"] == "accepted":
                print(f"bought {order_size} of {asset} at {bid_price}")
                sell_order = make_order(asset, order_size, "sell", "limit", "fok", bid_price)
                if sell_order["status"] == "accepted":
                    print(f"sold {order_size} of {asset} at {ask_price}")

Now to test this out, we just make a quick change to our websocket.

In [10]:
def on_open(ws):
    # we have to send an authentication message
    auth_data = {
        "action": "authenticate",
        "data": {
            "key_id": f"{KEY}",
            "secret_key": f"{SECRET_KEY}"
        }
    }
    # authenticate our connection
    ws.send(json.dumps(auth_data))

    channel_data = {
        "action": "listen",
        "data": {"streams": STOCKS}  # streams is what channels we want to subscribe to
    }

    ws.send(json.dumps(channel_data))
    
    
def on_message(ws, message):
    analyze(ws, message)
    
def on_close(ws):
    print("closed connection")

In [11]:
ws = websocket.WebSocketApp(SOCKET_ENDPOINT, on_open=on_open, on_message=on_message, on_close=on_close)
ws.run_forever()

closed connection


True

Now when this runs we notice that we very rarely got orders filled.  We correctly identified when to purchase orders, but by the time our order was processed by the exchange there were no longer anymore shares to be purhcased near the bid price.  This means that we were simply too slow.

And this is basically as far as you can get with market making as a retail trader.  We're operating on the millisecond latency scale, but high frequency trading firms operate on the nanosecond scale, meaning they're roughly a million times faster than we are.  This basically means that they place servers to run their code literally across the street from places like the NYSE or sometimes even inside the exchanges themselves, and with all this infastructure there is almost no chance you'll beat a high frequency trader to the exchange.  

Speedups also come from code.  Python is notoriously slow, meaning that quants oftentimes test ideas and benchmark them in python since it's easy to use, but production algotrading code is almost entirely written in C or C++, and sometimes Assembley to get the fastest code.  

Same is true for arbitrage, which is free money if you have fast enough computers.